In [23]:
import pandas as pd

In [24]:
df = pd.read_csv("whole_quarter_cities_venues.csv")
df.drop_duplicates(inplace=True)

Attraction name = artist(s), format of artist|artist, split so that matching can happen
expand df such that each row contains only 1 artist, event id + artist will be the primary key

In [25]:
exploded_series = df['ATTRACTION_NAME'].str.split('|').explode()
df_expanded = df.reindex(exploded_series.index)
df_expanded['individual_artist'] = exploded_series.str.lower()

In [26]:
spotify_df = pd.read_csv("spotify_artist_data_2023.csv")
spotify_df.columns

Index(['id', 'name', 'artist_popularity', 'artist_genres', 'followers',
       'genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5',
       'genre_6'],
      dtype='object')

popularity score assigned to concert is popularity of most popular artist

In [27]:
spotify_df["lower_name"] = spotify_df["name"].str.lower()
merged = df_expanded.merge(spotify_df, how="inner", left_on="individual_artist", right_on="lower_name")
just_popular = merged[["EVENT_ID", "artist_popularity"]].groupby("EVENT_ID").max()
df_with_popularity = merged.merge(
    just_popular, 
    how="inner", 
    left_on=["EVENT_ID", "artist_popularity"],
    right_on=["EVENT_ID", "artist_popularity"]
)
df_with_popularity.shape

(823, 47)

In [30]:
df_with_popularity.to_csv("823rows.csv")